# Snakemake

[Source](http://slowkow.com/notes/snakemake-tutorial/)

In [ ]:
# Make a fake genome:
!touch genome.fa

# Make some fake data:
!mkdir fastq
!touch fastq/Sample1.R1.fastq.gz fastq/Sample1.R2.fastq.gz
!touch fastq/Sample2.R1.fastq.gz fastq/Sample2.R2.fastq.gz

In [1]:
%%writefile Snakefile
SAMPLES = ['Sample1', 'Sample2']

rule all:
    input:
        expand('{sample}.txt', sample=SAMPLES)

rule quantify_genes:
    input:
        genome = 'genome.fa',
        r1 = 'fastq/{sample}.R1.fastq.gz',
        r2 = 'fastq/{sample}.R2.fastq.gz'
    output:
        '{sample}.txt'
    shell:
        'echo {input.genome} {input.r1} {input.r2} > {output}'

Overwriting Snakefile


In [5]:
## dry run
!snakemake -np

Nothing to be done.


In [2]:
!snakemake

Nothing to be done.


In [3]:
!head Sample?.txt

==> Sample1.txt <==
genome.fa fastq/Sample1.R1.fastq.gz fastq/Sample1.R2.fastq.gz

==> Sample2.txt <==
genome.fa fastq/Sample2.R1.fastq.gz fastq/Sample2.R2.fastq.gz


For more see source.

In [ ]:
## Another example (Will not run)

In [9]:
"""
    This is a basic framework for processing of of viral genomes, currently
    tailored for EBOV. Some generalization work needed to expand this
    to generic viral genomes with an arbitrary number of segments/chromosomes.

    Make copies of this Snakefile and config.yaml to your analysis directory and
    customize as needed.
    
    https://github.com/broadinstitute/viral-ngs/blob/master/pipes/Snakefile
"""

__author__ = 'Daniel Park <dpark@broadinstitute.org>'

import os

configfile: "config.yaml"
pipesDir = os.path.join(os.path.expanduser(config['bin_dir']), 'pipes', 'rules')

include: os.path.join(pipesDir, 'common.rules')
set_env_vars()

include: os.path.join(pipesDir, 'demux.rules')
include: os.path.join(pipesDir, 'hs_deplete.rules')
include: os.path.join(pipesDir, 'assembly.rules')
include: os.path.join(pipesDir, 'metagenomics.rules')
include: os.path.join(pipesDir, 'interhost.rules')
include: os.path.join(pipesDir, 'ncbi.rules')
include: os.path.join(pipesDir, 'intrahost.rules')
include: os.path.join(pipesDir, 'reports.rules')

samples_assembly = list(read_samples_file(config["samples_assembly"]))
samples_metagenomics = list(read_samples_file(config["samples_metagenomics"]))

all_inputs = [
    # create final assemblies for all samples
    expand("{data_dir}/{subdir}/{sample}.fasta",
           data_dir=config["data_dir"], subdir=config["subdirs"]["assembly"],
           sample=samples_assembly),
    # create BAMs of aligned reads to own consensus and to common ref
    expand("{data_dir}/{subdir}/{sample}.bam",
           data_dir=config["data_dir"], subdir=config["subdirs"]["align_self"],
           sample=samples_assembly),
    # intrahost variant calling
    config["data_dir"]+'/'+config["subdirs"]["intrahost"] +'/isnvs.vcf.gz',
    # create summary reports
    expand("{reports_dir}/summary.fastqc.{adjective}.txt",
           reports_dir=config["reports_dir"],
           adjective=['raw', 'cleaned', 'taxfilt', 'align_to_self'])
]

if config.get('spikeins_db'):
    all_inputs.append(config["reports_dir"]+'/summary.spike_count.txt')
if config.get('kraken_db'):
    all_inputs.extend(
        expand(os.path.join(config["data_dir"], config["subdirs"]["metagenomics"],
                    "{sample}.raw.kraken.report"), sample=samples_metagenomics))

rule all:
    input: all_inputs
    params: LSF="-N"
    run:
            if "job_profiler" in config:
                print("running report on all job runs")
                shell("{config[job_profiler]} {config[log_dir]} {config[reports_dir]}/summary.job_stats.txt")
            print("echo all done!")

rule clean:
    params: LSF="-N"
    shell: "rm -rf {config[tmp_dir]}/* .snakemake/tmp.*"

SyntaxError: invalid syntax (<ipython-input-9-aa71f0f55f82>, line 16)

## Currently not working

In [4]:
!snakemake --forceall --dag | dot -Tpng | display

/bin/sh: display: command not found
dyld: Library not loaded: /usr/local/lib/libltdl.7.dylib
  Referenced from: /Users/squiresrb/anaconda/envs/snakemake/bin/dot
  Reason: image not found
Exception ignored in: <_io.TextIOWrapper name='<stdout>' mode='w' encoding='UTF-8'>
BrokenPipeError: [Errno 32] Broken pipe


In [ ]:
conda install libtool
conda install snakemake